# ANN Model Benchmark

## Setup

In [1]:
import pandas as pd
import os
import tensorflow as tf
import pickle
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

from tensorflow_addons.callbacks import TQDMProgressBar

2024-04-09 11:37:21.957227: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-09 11:37:21.977800: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-09 11:37:21.977820: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-09 11:37:21.978363: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-09 11:37:21.981951: I tensorflow/core/platform/cpu_feature_guar

In [2]:
os.makedirs(name=f"data/models/eval/predict", exist_ok=True)
os.makedirs(name=f"data/models/eval/metrics", exist_ok=True)

In [3]:
MAX_EPOCHS = 750
DENSE_ACTIVATION = "elu"
DENSE_LAYERS = 3
DENSE_UNITS = {
    1: 230,
    2: 54,
    3: 6,
}
DENSE_DROPOUT = {
    1: 0.01,
    2: 0.01,
}
REG_BETA = 0.01
LEARNING_RATE = 0.0001091143
MIN_LEARNING_RATE = 0.275 * LEARNING_RATE
BATCH_SIZE = 448

In [4]:
# get preprocessed data
df = pd.read_pickle("data/preprocessed_data/ANN/dataset.pkl").query("subsequent_flag_1 == 0")

with open(f"data/preprocessed_data/ANN/columns.pkl", "rb") as file:
    y_col, x_cols = pickle.load(file)
df_cal = df.query("partition in ('train', 'validation') and train_partition == 'calibration'")
df_tune = df.query("partition in ('train', 'validation') and train_partition == 'tunning'")

In [5]:
def create_model(
    input_dims: int,
    dense_layers: int,
    activation_function: str,
    dense_units: dict,
    dense_dropout: dict,
    reg_beta: float,
    learning_rate: float,
    random_seed: int = None
) -> tf.keras.models.Sequential:

    # clear keras session
    tf.keras.backend.clear_session()
    
    # weights L2 regularization (all layers)
    kernel_reg = tf.keras.regularizers.l2(reg_beta)

    # weights initialisation (all layers)
    kernel_init = tf.keras.initializers.he_normal(seed=random_seed)

    # model architecture
    model = tf.keras.Sequential()
    
    model.add(
        tf.keras.layers.Input(
            name="input",
            shape=(input_dims,),
        )
    )
    
    # dense layers
    for layer in range(1, dense_layers+1):
        model.add(
            tf.keras.layers.Dense(
                name=f"dense_{layer}",
                units=dense_units[layer],
                kernel_initializer=kernel_init,
                kernel_regularizer=kernel_reg,
                activation=activation_function
            )
        )
        if layer != dense_layers:
            model.add(
                tf.keras.layers.Dropout(
                    name=f"dropout_{layer}",
                    rate=dense_dropout[layer]
                    
                )
            )
    
    # output layer
    model.add(     
        tf.keras.layers.Dense(
            name="output",
            units=1,
            kernel_initializer=kernel_init,
            kernel_regularizer=kernel_reg,
            activation="linear"
        )
    )
    
    # compile model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate),
        loss="mse",
        metrics=["mse"]
    )
    
    return model


def create_callbacks( 
    lr_min: float, 
) -> list:
    
    callbacks = []
    
    # reduce learning rate dynamically
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
        patience=25,
        factor=0.5,
        min_lr=lr_min,
        monitor="val_loss",
        verbose=0
    )
    callbacks.append(reduce_lr)

    # early stopping criteria
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        min_delta=1e-4,  
        patience=50,
        mode="min",
        restore_best_weights=True,
    )
    callbacks.append(early_stopping)

    # progress bar during training
    progress_bar = TQDMProgressBar(show_epoch_progress=False)
    callbacks.append(progress_bar)
    
    return callbacks

In [6]:
def calculate_metrics(y_true: np.ndarray, y_pred: np.ndarray) -> dict:
    """Calculate SEP, RMSE, Bias, and RPD of predictions

    """
    n = y_true.shape[0]
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    y_error = y_true - y_pred
    mean_error = np.mean(y_error)
    std_error = np.sqrt(np.square(y_error - mean_error).sum() / (n-1))
    std_true = np.sqrt(np.square(y_true - y_true.mean()).sum() / (n-1))
    return {
        # number of samples
        "n": len(y_true),
        
        # calculate r-squared (R2)
        "r2": r2_score(y_true, y_pred),

        # calculate root mean square error (RMSE)
        "rmse": rmse,

        # calculate standard error of prediction (SEP)
        "sep": std_error,

        # calculate bias
        "bias": mean_error,

        # calculate ratio of performance to deviation (RPD)
        "rpd": std_true / std_error,
    }

In [7]:
test_sets = {
    "training": "partition in ('train', 'validation')",
    "training_calibration": "partition in ('train', 'validation') and train_partition == 'calibration'",
    "training_tuning": "partition in ('train', 'validation') and train_partition == 'tunning'",
    "holdout": "partition == 'holdout'",
    "season 2020": "season == 2020",
    "season 2021": "season == 2021",
    
}

all_metrics = []

In [8]:
for random_seed in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
    # model initialization and compile
    model = create_model(
        input_dims=len(x_cols),
        activation_function=DENSE_ACTIVATION,
        dense_layers=DENSE_LAYERS,
        dense_units=DENSE_UNITS,
        dense_dropout=DENSE_DROPOUT,
        reg_beta=REG_BETA,
        learning_rate=LEARNING_RATE,
        random_seed=random_seed
    )

    # define callbacks
    callbacks = create_callbacks(
        lr_min=MIN_LEARNING_RATE,
    )

    # train model 
    history = model.fit(
        x=df_cal[x_cols],
        y=df_cal[y_col],
        batch_size=BATCH_SIZE,
        epochs=MAX_EPOCHS,
        validation_data=(df_tune[x_cols], df_tune[y_col]),
        callbacks=callbacks,
        verbose=0
    )

    # save model
    model.save(f"data/models/eval/ann_rs{random_seed}.model.keras")
    
    # make and save predictions
    df_pred = df.copy()
    df_pred["y_true"] = df_pred["dry_matter"]
    df_pred["y_pred"] = model.predict(df[x_cols])
    df_pred.to_pickle(f"data/models/eval/predict/ann_rs{random_seed}.pkl")
    
    
    for test_set, query in test_sets.items():
        test_partition = df_pred.query(query)
        metrics = calculate_metrics(
            y_true=test_partition["y_true"], 
            y_pred=test_partition["y_pred"]
        )
        metrics["model"] = f"ann_rs{random_seed}"
        metrics["test_set"] = test_set
        metrics["query"] = query
        all_metrics.append(metrics)

    metrics = pd.DataFrame(all_metrics)
    metrics.to_csv(f"data/models/eval/metrics/ann.csv")

2024-04-09 11:37:28.399584: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-09 11:37:28.473718: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-09 11:37:28.473943: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

2219/2219 [==============================] - 3s 1ms/step


Training:  75%|███████▍   561/750 ETA: 02:20s,   1.34epochs/s


2219/2219 [==============================] - 3s 2ms/step


Training:  81%|████████   607/750 ETA: 01:47s,   1.33epochs/s


...

2219/2219 [==============================] - 3s 1ms/step


Training:  79%|███████▉   594/750 ETA: 01:50s,   1.41epochs/s


2219/2219 [==============================] - 3s 1ms/step


Training:  75%|███████▌   566/750 ETA: 02:21s,   1.30epochs/s


2219/2219 [==============================] - 3s 2ms/step


In [9]:
pd.DataFrame(all_metrics)

,n,r2,rmse,sep,bias,rpd,model,test_set,query
0,68009,0.936771,0.618765,0.618720,0.007818,3.977179,cnn_rs1,training,"partition in ('train', 'validation')"
1,54341,0.940032,0.606604,0.606319,0.018793,4.085527,cnn_rs1,training_calibration,"partition in ('train', 'validation') and train..."
2,13668,0.922892,0.664916,0.663975,-0.035815,3.606461,cnn_rs1,training_tuning,"partition in ('train', 'validation') and train..."
3,2996,0.763025,1.203980,1.113006,0.459563,2.222507,cnn_rs1,holdout,partition == 'holdout'
4,2594,0.787611,1.114809,1.038043,0.407038,2.330788,cnn_rs1,season 2020,season == 2020
5,402,0.522931,1.668470,1.466817,0.798494,1.648894,cnn_rs1,season 2021,season == 2021
6,68009,0.936788,0.618681,0.617905,0.031069,3.982422,cnn_rs2,training,"partition in ('train', 'validation')"
7,54341,0.939888,0.607332,0.605853,0.042436,4.088670,cnn_rs2,training_calibration,"partition in ('train', 'validation') and train..."
8,13668,0.923594,0.661884,0.661757,-0.014127,3.618546,cnn_rs2,training_tuning,"partition in ('train', 'validation') and train..."
9,2996,0.711893,1.327533,1.295681,-0.290023,1.909161,cnn_rs2,holdout,partition == 'holdout'
